# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

Импортируем библиотеки, с которыми будем работать

In [1]:
import pandas as pd
import numpy as np

import seaborn as sbn
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings('ignore')

Сохраним данные в переменную `df` и посмотрим первые 5 строк датасета:

In [2]:
try:
    df = pd.read_csv('insurance.csv')
except:
    df = pd.read_csv('/datasets/insurance.csv')
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Посмотрим общую информацию о данных:

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Посмотрим описательную статистику по каждому столбцу датафрейма:

In [4]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


В таблице данные клиентов страховой компании "Хоть потоп" 5000 объектов. Пропущенных значений нет. Проверим наличие дубликатов данных.

In [5]:
df.duplicated().sum()

153

В 5000 наблюдений содержится 153 дубликата. Поскольку точно идентифицировать каждого клиента мы не можем по имеющимся данным, дубликаты удалять не будем - существенного влияния на модель эти данные не окажут.  
Выделим в исходных данных:
* **признаки**: пол, возраст и зарплата застрахованного, количество членов его семьи. 
* **целевой признак**: количество страховых выплат клиенту за последние 5 лет.

In [6]:
features = df.drop(['Страховые выплаты'], axis=1)
target = df['Страховые выплаты']

print(features.shape)
print(target.shape)

(5000, 4)
(5000,)


Разделим данные на обучающую и тестовую выборки в пропорции 3:1 соответственно (75% на обучающию и 25% данных тестовую выборку)

In [7]:
df_train, df_test = train_test_split(df, test_size=0.25, random_state=12345)
print('Размер обучающей выборки', df_train.shape)
print('Размер тестовой выборки', df_test.shape)

Размер обучающей выборки (3750, 5)
Размер тестовой выборки (1250, 5)


Составим таблицы признаков и целевого признака для каждой выборки.

In [8]:
features_train = df_train.drop(['Страховые выплаты'], axis=1)
target_train = df_train['Страховые выплаты']
features_test = df_test.drop(['Страховые выплаты'], axis=1)
target_test = df_test['Страховые выплаты']

### Вывод
В ходе предобработки данных не выявили аномалий и пропущенных значений. Обнаруженные дубликаты данных оставили без изменений, поскольку значительного влияния на модель эти данные не окажут.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.







Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** При умножении признаков на обратимую матрицу, качество линейной регрессии не меняется.

**Обоснование:** Умножим матрицу признаков на обратимую матрицу $Р$ и полученную матрицу назовем $М$:
$$
M = XP
$$

Подставим в формулу обучения новую матрицу:
$$
w_1 = (M^T M) ^{-1} M^T y
$$

$$
w_1 = ((XP)^T XP) ^{-1} (XP)^T y
$$

Преобразуем, используя свойства матриц:
$$
w_1 = (P^T X^T X P)^{-1} P^T X^T y
$$

Используем правило для квадратных матриц $(AB)^{-1} = B^{-1} A^{-1}$
получаем:

$$
w_1 = P^{-1}(X^T X)^{-1}(P^T)^{-1} P^T X^T y
$$

При умножении матрицы на обратную матрицу, получаем единичную матрицу:
$$
w_1 = P^{-1}(X^T X)^{-1} E X^T y
$$

$$
w_1 = P^{-1}(X^T X)^{-1} X^T y
$$

$$
w_1 = P^{-1} w
$$

Подставим в формулу предсказаний новую матрицу $M$ и $w_1$, вычислим предсказания $a_1$:
$$
a_1 = M w_1 = XP P^{-1} w
$$

При умножении матрицы на обратную матрицу снова получаем единичную матрицу: 
$$
a_1 = X w
$$

Таким образом, предсказания $a_1$ для матрицы признаков, умноженной на обратимую матрицу $P$ равны предсказаниям $a$:
$$
a_1 = a
$$

## Алгоритм преобразования

**Алгоритм**
1. Создать случайную обратимую матрицу $P$ с помощью `numpy.random.normal()` размером `4*4` по количеству признаков основной матрицы, которую будем преобразовывать.
2. Проверить сгенерированную матрицу на обратимость.
3. Преобразовать исходные выборки в матрицы атрибутом `values`.
4. Зашифровать данные - умножить матрицу признаков $X$ на матрицу $P$.
5. Для расшифровки данных умножить полученную матрицу на матрицу $P^ {-1}$, обратную случайной.
6. Преобразовать полученную матрицу в датафрейм и убедиться, что данные не изменились.
7. Инициировать и обучить модель линейной регрессии на исходных данных.
8. Получить предсказания и оценить качество модели метрикой R2 на исходных данных.
9. Инициировать и обучить модель линейной регрессии на зашифрованных данных.
10. Получить предсказания и оценить качество модели метрикой R2 на зашифрованных данных.
11. Сравнить полученные метрики и сделать вывод.

**Обоснование**
$$
XP P^{-1} = X
$$

При умножении матрицы на обратную матрицу получается единичная матрица.

## Проверка алгоритма

Запрограммируем предложенный алгоритм - для начала создадим случайную квадратную матрицу P размерности `4*4` по количеству признаков:

In [9]:
p = np.random.normal(size = (4,4))
p

array([[ 1.42942767,  0.38918985, -0.69361109,  0.95124111],
       [ 0.8125933 , -0.20193221,  0.00679561, -0.45310964],
       [-0.35462442,  1.63188015, -1.16682621,  1.53817192],
       [ 0.29835684, -0.68307704,  0.3756828 , -1.425255  ]])

Проверим матрицу P на обратимость - умножим её на обратную матрицу:

In [10]:
np.round(p @ np.linalg.inv(p))

array([[ 1.,  0., -0., -0.],
       [-0.,  1.,  0., -0.],
       [ 0., -0.,  1.,  0.],
       [-0.,  0., -0.,  1.]])

Получили единичную матрицу, значит матрица Р - обратима, можем использовать её для проверки нашего алгоритма. Создадим матрицы из исходных выборок атрибутом `values`.

In [11]:
features_train_x = features_train.values
features_test_x = features_test.values

print(features_train_x.shape)
print(features_test_x.shape)

(3750, 4)
(1250, 4)


Зашифруем данные - умножим матрицу признаков на случайную матрицу :

In [12]:
features_train_m = features_train_x @ p
features_test_m = features_test_x @ p

Для расшифровки данных умножним новую матрицу признаков на обратную случайной:

In [13]:
features_train_new_x = features_train_m @ (np.linalg.inv(p))
features_test_new_x = features_test_m @ (np.linalg.inv(p))

Преобразуем полученную матрицу в датафрейм:

In [14]:
new_x = pd.DataFrame(features_train_new_x, columns=features_train.columns)
np.round(new_x)

,Пол,Возраст,Зарплата,Члены семьи
0,1.0,43.0,36200.0,1.0
1,1.0,34.0,57600.0,0.0
2,0.0,32.0,41100.0,1.0
3,0.0,36.0,45100.0,1.0
4,0.0,33.0,50600.0,2.0
...,...,...,...,...
3745,0.0,42.0,32100.0,0.0
3746,0.0,28.0,22700.0,4.0
3747,1.0,41.0,44700.0,1.0
3748,0.0,22.0,50100.0,4.0


После применения предложенного алгоритма получили исходную таблицу с данными. Алгоритм работает. Проверим, что качество линейной регрессии не отличается до и после преобразования, применим метрику R2 сначала на исходных данных, затем на зашифрованных.

In [15]:
model = LinearRegression()
model.fit(features_train_x, target_train)
predictions_x = model.predict(features_test_x)
r2_score_x = r2_score(target_test, predictions_x)
print('Качество линейной регресси до преобразования:', r2_score_x)

Качество линейной регресси до преобразования: 0.43522757127026546


In [16]:
model = LinearRegression()
model.fit(features_train_m, target_train)
predictions_m = model.predict(features_test_m)
r2_score_m = r2_score(target_test, predictions_m)
print('Качество линейной регресси после преобразования:', r2_score_m)

Качество линейной регресси после преобразования: 0.4352275712713949


Как видим, качество линейной регрессии до и после преобразования данных не изменилось. 

## Общий вывод

Объект нашей работы - данные клиентов страховой компании "Хоть потоп".

В ходе предобработки мы не обнаружили серьезных проблем, пропущенных значений и ошибок в данных. Разделили данные на обучающую и тестовую выборку.

Мы дали ответ на вопрос об умножении матриц, обосновали ответ формулами.

Для решения поставленной задачи мы предложили и обосновали алгоритм шифрования данных. Проверили работу алгоритма - зашифровали и расшифровали данные после преобразования случайной матрицей. Сравнили качество моделей до и после преобразования с помощью метрики R2, качество моделей не изменилось - предложенный алгоритм работает корректно.

Подведём итог нашей работы - рекомендация для страховой компании "Хоть потоп":
* обратить внимание и возможно использовать предложенный в ходе работы алгоритм преобразования для защиты данных клиентов.